In [1]:
from pathlib import Path

import numpy as np
import tree
import scipy.io as sio
from deeprte.data.tool.matlab import mat_loader


In [2]:
source_dir = Path("/workspaces/deeprte/rte_data/matlab/eval-data/scattering-kernel/0311")
data_name_list = ["test_bc1.mat"]

In [3]:
BATCH_FEAT_LIST = [
    "sigma_a",
    "sigma_t",
    "psi_label",
    "psi_bc",
    "scattering_kernel",
]
def mat_loader(source_dir: str, data_name_list: list[str]) -> dict[str, np.ndarray]:
    data_list = []
    for filename in data_name_list:
        data_path = source_dir / filename
        mat_dict = sio.loadmat(data_path)
        if "scattering_kernel" not in mat_dict.keys():
            rng = np.random.default_rng(12345)
            num_sample = mat_dict["sigma_a"].shape[0]
            num_vec = mat_dict["ct"].shape[0]
            mat_dict["scattering_kernel"] = rng.uniform(
                0, 1, (num_sample, num_vec, num_vec)
            )
        data_list.append(mat_dict)
    data = data_list[0]

    for k in BATCH_FEAT_LIST:
        data[k] = np.concatenate([d[k] for d in data_list], axis=0)

    unused_keys = [k for k in data.keys() if k.startswith("__")]
    # for k in unused_keys
    for k in unused_keys:
        del data[k]

    return data

In [4]:
data = mat_loader(source_dir, data_name_list)

In [5]:
data.keys()

dict_keys(['ct', 'omega', 'omega_prime', 'phi', 'psi_bc', 'psi_label', 'r', 'rv', 'rv_prime', 'sigma_a', 'sigma_t', 'st', 'w_angle', 'x', 'y', 'scattering_kernel'])

In [7]:
def interpolate(data):
    list_omega = [data["omega_prime"][41*i:41*(i+1),:] for i in range(4)]
    list_omega = [(omega[1:]+omega[:-1])/2 for omega in list_omega]
    data["omega_prime"] = np.concatenate(list_omega, axis=0)

    phi = data["phi"]
    phi = (phi[:,:-1,:]+phi[:,1:,:])/2
    data["phi"] = (phi[:,:,:-1]+phi[:,:,1:])/2

    list_psi_bc = [data["psi_bc"][:,41*i:41*(i+1),:] for i in range(4)]
    list_psi_bc = [(psi_bc[:,1:,:]+psi_bc[:,:-1,:])/2 for psi_bc in list_psi_bc]
    data["psi_bc"] = np.concatenate(list_psi_bc, axis=-2)

    psi = data["psi_label"]
    psi = (psi[:,:-1,:,:]+psi[:,1:,:,:])/2
    data["psi_label"] = (psi[:,:,:-1,:]+psi[:,:,1:,:])/2

    list_rv_prime = [data["rv_prime"][41*i:41*(i+1),:,:] for i in range(4)]
    list_rv_prime = [(rv_prime[1:,:,:]+rv_prime[:-1,:,:])/2 for rv_prime in list_rv_prime]
    data["rv_prime"] = np.concatenate(list_rv_prime, axis=0)

    sigma_a = data["sigma_a"]
    sigma_a = (sigma_a[:,:-1,:]+sigma_a[:,1:,:])/2
    data["sigma_a"] = (sigma_a[:,:,:-1]+sigma_a[:,:,1:])/2

    sigma_t = data["sigma_t"]
    sigma_t = (sigma_t[:,:-1,:]+sigma_t[:,1:,:])/2
    data["sigma_t"] = (sigma_t[:,:,:-1]+sigma_t[:,:,1:])/2

    x = np.squeeze(data["x"])
    x = (x[1:]+x[:-1])/2
    data["x"] = x

    y = np.squeeze(data["y"])
    y = (y[1:]+y[:-1])/2
    data["y"] = y

    return data

In [8]:
data = interpolate(data)

In [9]:
tree.map_structure(lambda x: x.shape, data)

{'ct': (24, 1),
 'omega': (24, 1),
 'omega_prime': (160, 12),
 'phi': (10, 40, 40),
 'psi_bc': (10, 160, 12),
 'psi_label': (10, 40, 40, 24),
 'r': (41, 41, 2),
 'rv': (41, 41, 24, 4),
 'rv_prime': (160, 12, 4),
 'sigma_a': (10, 40, 40),
 'sigma_t': (10, 40, 40),
 'st': (24, 1),
 'w_angle': (24, 1),
 'x': (40,),
 'y': (40,),
 'scattering_kernel': (10, 24, 24)}

In [24]:
np.allclose(data["psi_bc"][9, 40:, :],  np.zeros((120, 12)))

True

In [10]:
def standardization(data):
    mu = np.mean(data)
    sigma = np.std(data)
    return (data - mu) / sigma


def normalization(data):
    _range = np.max(data) - np.min(data)
    return (data - np.min(data)) / _range

In [11]:
std_data = normalization(data["psi_label"])

In [12]:
np.max(std_data)

1.0

In [13]:
np.min(std_data)

0.0